In [1]:
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.colors as mcolors
import pandas as pd

In [2]:
import torch
import torch.nn.utils

import sys
sys.path.append(os.path.abspath('../segment-anything-2'))

from sklearn.model_selection import train_test_split
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor

In [3]:
torch.cuda.empty_cache()

# Fine Tuning

In [4]:
df = pd.DataFrame(
    {
        "ImageId": sorted(os.listdir('../data/raw/AneRBC-I/Anemic_individuals/Original_images') + os.listdir('../data/raw/AneRBC-I/Healthy_individuals/Original_images')),
        "MaskId": sorted(os.listdir('../data/raw/AneRBC-I/Anemic_individuals/Binary_segmented') + os.listdir('../data/raw/AneRBC-I/Healthy_individuals/Binary_segmented'))
    }
)

df

,ImageId,MaskId
0,001_a.png,001_a.png
1,001_h.png,001_h.png
2,002_a.png,002_a.png
3,002_h.png,002_h.png
4,003_a.png,003_a.png
...,...,...
995,498_h.png,498_h.png
996,499_a.png,499_a.png
997,499_h.png,499_h.png
998,500_a.png,500_a.png


In [5]:
images_a_dir = '../data/raw/AneRBC-I/Anemic_individuals/Original_images'
masks_a_dir = '../data/raw/AneRBC-I/Anemic_individuals/Binary_segmented'
images_h_dir = '../data/raw/AneRBC-I/Healthy_individuals/Original_images'
masks_h_dir = '../data/raw/AneRBC-I/Healthy_individuals/Binary_segmented'

# Load the train.csv file
train_df = df.copy()

# 600 for fine-tuning, 100 for validation and 300 for testing
train_df, test_df = train_test_split(train_df, test_size=0.3, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.142, random_state=42)

# Prepare the training data list
train_data = []
for index, row in train_df.iterrows():
   image_name = row['ImageId']
   mask_name = row['MaskId']

   # Append image and corresponding mask paths
   if image_name in os.listdir(images_a_dir):
        images_dir = images_a_dir
        masks_dir = masks_a_dir
   else:
        images_dir = images_h_dir
        masks_dir = masks_h_dir

   train_data.append({
       "image": os.path.join(images_dir, image_name),
       "annotation": os.path.join(masks_dir, mask_name)
   })


val_data = []
for index, row in val_df.iterrows():
   image_name = row['ImageId']
   mask_name = row['MaskId']

   if image_name in os.listdir(images_a_dir):
        images_dir = images_a_dir
        masks_dir = masks_a_dir
   else:
        images_dir = images_h_dir
        masks_dir = masks_h_dir

   # Append image and corresponding mask paths
   val_data.append({
       "image": os.path.join(images_dir, image_name),
       "annotation": os.path.join(masks_dir, mask_name)
   })

# Prepare the testing data list (if needed for inference or evaluation later)
test_data = []
for index, row in test_df.iterrows():
   image_name = row['ImageId']
   mask_name = row['MaskId']

   if image_name in os.listdir(images_a_dir):
        images_dir = images_a_dir
        masks_dir = masks_a_dir
   else:
        images_dir = images_h_dir
        masks_dir = masks_h_dir

   # Append image and corresponding mask paths
   test_data.append({
       "image": os.path.join(images_dir, image_name),
       "annotation": os.path.join(masks_dir, mask_name)
   })


In [6]:
print(f"Training data: {len(train_data)}")
print(f"Validation data: {len(val_data)}")
print(f"Testing data: {len(test_data)}")

Training data: 600
Validation data: 100
Testing data: 300


In [7]:
sam2_checkpoint = "../segment-anything-2/sam2_hiera_small.pt"
model_cfg = "C:/Users/saksh/OneDrive/Documents/Duke Academics/Spring 2025/Deep Learning/CV-Module-Project/segment-anything-2/sam2/configs/sam2/sam2_hiera_s.yaml"

sam2_model = build_sam2(model_cfg, sam2_checkpoint, device="cuda")
predictor = SAM2ImagePredictor(sam2_model)

# Training

Image Chosen: ../data/raw/AneRBC-I/Healthy_individuals/Original_images\050_h.png


In [7]:
def read_batch(data, visualize_data=False):
   # Select a random entry
   ent = data[np.random.randint(len(data))]

   # Get full paths
   Img = cv2.imread(ent["image"])[..., ::-1]  # Convert BGR to RGB
   ann_map = cv2.imread(ent["annotation"], cv2.IMREAD_GRAYSCALE)  # Read annotation as grayscale

   if visualize_data:
       print(f"Image Chosen: {ent['image']}")

   if Img is None or ann_map is None:
       print(f"Error: Could not read image or mask from path {ent['image']} or {ent['annotation']}")
       return None, None, None, 0

   # Resize image and mask
   r = np.min([1024 / Img.shape[1], 1024 / Img.shape[0]])  # Scaling factor
   Img = cv2.resize(Img, (int(Img.shape[1] * r), int(Img.shape[0] * r)))
   ann_map = cv2.resize(ann_map, (int(ann_map.shape[1] * r), int(ann_map.shape[0] * r)), interpolation=cv2.INTER_NEAREST)

   # Convert to binary mask (0 for background, 1 for masked region)
   binary_mask = np.where(ann_map == 255, 0, 1).astype(np.uint8)

   # Erode the binary mask to avoid boundary points
   eroded_mask = cv2.erode(binary_mask, np.ones((5, 5), np.uint8), iterations=1)

   # Get all coordinates inside the eroded mask (where the mask value is 1)
   coords = np.argwhere(eroded_mask > 0)

   # Select random points from the eroded mask
   num_points = 100  # Number of points to select

   # Randomly sample points from `coords` if there are enough points
   if len(coords) > num_points:
        points = coords[np.random.choice(len(coords), num_points, replace=False)]
   else:
        points = coords  # If fewer points exist, use all of them

   # Reformat points to (x, y) from (y, x)
   points = np.array([[p[1], p[0]] for p in points])

   if visualize_data:
        # Plotting the images and points
        plt.figure(figsize=(15, 5))

        # Original Image
        plt.subplot(1, 3, 1)
        plt.title('Original Image')
        plt.imshow(Img)
        plt.axis('off')

        # Segmentation Mask (binary_mask)
        plt.subplot(1, 3, 2)
        plt.title('Binarized Mask')
        plt.imshow(binary_mask, cmap='gray')
        plt.axis('off')

        # Mask with Points in Different Colors
        plt.subplot(1, 3, 3)
        plt.title('Binarized Mask with Points')
        plt.imshow(binary_mask, cmap='gray')

        # Plot points in different colors
        colors = list(mcolors.TABLEAU_COLORS.values())
        for i, point in enumerate(points):
            plt.scatter(point[0], point[1], c=colors[i % len(colors)], s=10, label=f'Point {i+1}')  # Corrected to plot y, x order

        # plt.legend()
        plt.axis('off')

        plt.tight_layout()
        plt.show()

   binary_mask = np.expand_dims(binary_mask, axis=-1)  # Now shape is (1024, 1024, 1)
   binary_mask = binary_mask.transpose((2, 0, 1))
   points = np.expand_dims(points, axis=1)

   # Return the image, binarized mask, points, and number of masks
   return Img, binary_mask, points, num_points

# Visualize the data
Img1, masks1, points1, num_masks = read_batch(train_data, visualize_data=True)

In [8]:
# Configuring Hyperparameters
# Train mask decoder.
predictor.model.sam_mask_decoder.train(True)

# Train prompt encoder.
predictor.model.sam_prompt_encoder.train(True)

# Configure optimizer.
optimizer=torch.optim.AdamW(params=predictor.model.parameters(), lr=0.0001, weight_decay=1e-4) #1e-5, weight_decay = 4e-5

# Mix precision: more memory-efficient training strategy
scaler = torch.cuda.amp.GradScaler()

# No. of steps to train the model.
NO_OF_STEPS = 5 # @param

# Fine-tuned model name.
FINE_TUNED_MODEL_NAME = "fine_tuned_sam2_600_100_300_tvt"

C:\Users\saksh\AppData\Local\Temp\ipykernel_25208\3224331296.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [18]:
torch.cuda.empty_cache()

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [10]:
def dice_loss(y_true, y_pred, smooth=1e-6):
    """
    Compute the Dice loss between y_true and y_pred.
    y_true: Ground truth mask (binary, 0 or 1)
    y_pred: Model output logits (before sigmoid)
    """
    y_pred = torch.sigmoid(y_pred)  # Convert logits to probabilities
    intersection = (y_true * y_pred).sum(dim=(1, 2))  # Sum over spatial dimensions
    union = y_true.sum(dim=(1, 2)) + y_pred.sum(dim=(1, 2))  # Sum over spatial dimensions

    dice = (2. * intersection + smooth) / (union + smooth)  # Dice coefficient
    return 1 - dice.mean()  # Dice loss


In [11]:
def validate_model(predictor, val_data):

    total_val_loss = 0
    total_val_iou = 0
    num_batches = 0
    criterion = torch.nn.BCEWithLogitsLoss()

    with torch.no_grad():  # No gradient updates
        for index in range(len(val_data)):
            image, mask, input_point, num_masks = read_batch(val_data, visualize_data=False)
            if image is None or mask is None or num_masks == 0:
                continue

            input_label = np.ones((num_masks, 1))
            if not isinstance(input_point, np.ndarray) or not isinstance(input_label, np.ndarray):
                continue

            predictor.set_image(image)
            mask_input, unnorm_coords, labels, unnorm_box = predictor._prep_prompts(
                input_point, input_label, box=None, mask_logits=None, normalize_coords=True
            )
            if unnorm_coords is None or labels is None or unnorm_coords.shape[0] == 0 or labels.shape[0] == 0:
                continue

            sparse_embeddings, dense_embeddings = predictor.model.sam_prompt_encoder(
                points=(unnorm_coords, labels), boxes=None, masks=None,
            )

            batched_mode = unnorm_coords.shape[0] > 1
            high_res_features = [feat_level[-1].unsqueeze(0) for feat_level in predictor._features["high_res_feats"]]
            low_res_masks, prd_scores, _, _ = predictor.model.sam_mask_decoder(
                image_embeddings=predictor._features["image_embed"][-1].unsqueeze(0),
                image_pe=predictor.model.sam_prompt_encoder.get_dense_pe(),
                sparse_prompt_embeddings=sparse_embeddings,
                dense_prompt_embeddings=dense_embeddings,
                multimask_output=True,
                repeat_image=batched_mode,
                high_res_features=high_res_features,
            )
            prd_masks = predictor._transforms.postprocess_masks(low_res_masks, predictor._orig_hw[-1])

            gt_mask = torch.tensor(mask.astype(np.float32)).cuda()
            prd_mask = torch.sigmoid(prd_masks[:, 0])

            if gt_mask.shape[0] == 1 and prd_masks.shape[0] > 1:
                gt_mask = gt_mask.expand(prd_masks.shape[0], -1, -1)

            # Compute loss
            seg_bce_loss = criterion(prd_masks[:, 0], gt_mask)  
            seg_dice_loss = dice_loss(gt_mask, prd_masks[:, 0])  
            seg_loss = seg_bce_loss + seg_dice_loss  

            inter = (gt_mask * (prd_mask > 0.5)).sum(1).sum(1)
            iou = inter / (gt_mask.sum(1).sum(1) + (prd_mask > 0.5).sum(1).sum(1) - inter)

            total_val_loss += seg_loss.item()
            total_val_iou += np.mean(iou.cpu().detach().numpy())
            num_batches += 1

    avg_val_loss = total_val_loss / num_batches
    avg_val_iou = total_val_iou / num_batches

    print(f"Validation Loss: {avg_val_loss:.4f}, Validation IoU: {avg_val_iou:.4f}")
    return avg_val_loss, avg_val_iou

In [14]:
# # Initialize scheduler
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=500, gamma=0.2) # 500 , 250, gamma = 0.1
# accumulation_steps = 2  # Number of steps to accumulate gradients before updating

# for step in range(1, NO_OF_STEPS + 1):
#    with torch.cuda.amp.autocast():
#        image, mask, input_point, num_masks = read_batch(train_data, visualize_data=False)
#        if image is None or mask is None or num_masks == 0:
#            continue

#        input_label = np.ones((num_masks, 1))
#        if not isinstance(input_point, np.ndarray) or not isinstance(input_label, np.ndarray):
#            continue

#        if input_point.size == 0 or input_label.size == 0:
#            continue

#        predictor.set_image(image)
#        mask_input, unnorm_coords, labels, unnorm_box = predictor._prep_prompts(input_point, input_label, box=None, mask_logits=None, normalize_coords=True)
#        if unnorm_coords is None or labels is None or unnorm_coords.shape[0] == 0 or labels.shape[0] == 0:
#            continue

#        sparse_embeddings, dense_embeddings = predictor.model.sam_prompt_encoder(
    #        points=(unnorm_coords, labels), boxes=None, masks=None,
    #    )

    #    batched_mode = unnorm_coords.shape[0] > 1
    #    high_res_features = [feat_level[-1].unsqueeze(0) for feat_level in predictor._features["high_res_feats"]]
    #    low_res_masks, prd_scores, _, _ = predictor.model.sam_mask_decoder(
    #        image_embeddings=predictor._features["image_embed"][-1].unsqueeze(0),
    #        image_pe=predictor.model.sam_prompt_encoder.get_dense_pe(),
    #        sparse_prompt_embeddings=sparse_embeddings,
    #        dense_prompt_embeddings=dense_embeddings,
    #        multimask_output=True,
    #        repeat_image=batched_mode,
    #        high_res_features=high_res_features,
    #    )
    #    prd_masks = predictor._transforms.postprocess_masks(low_res_masks, predictor._orig_hw[-1])

    #    gt_mask = torch.tensor(mask.astype(np.float32)).cuda()
    #    prd_mask = torch.sigmoid(prd_masks[:, 0])
    #    seg_loss = (-gt_mask * torch.log(prd_mask + 0.000001) - (1 - gt_mask) * torch.log((1 - prd_mask) + 0.00001)).mean()

    #    inter = (gt_mask * (prd_mask > 0.5)).sum(1).sum(1)
    #    iou = inter / (gt_mask.sum(1).sum(1) + (prd_mask > 0.5).sum(1).sum(1) - inter)
    #    score_loss = torch.abs(prd_scores[:, 0] - iou).mean()
    #    loss = seg_loss + score_loss * 0.05

    #    # Apply gradient accumulation
    #    loss = loss / accumulation_steps
    #    scaler.scale(loss).backward()

    #    # Clip gradients
    #    torch.nn.utils.clip_grad_norm_(predictor.model.parameters(), max_norm=1.0)

    #    if step % accumulation_steps == 0:
    #        scaler.step(optimizer)
    #        scaler.update()
    #        predictor.model.zero_grad()

    #    # Update scheduler
    #    scheduler.step()

    #    if step % 500 == 0:
    #        FINE_TUNED_MODEL = FINE_TUNED_MODEL_NAME + "_" + str(step) + ".torch"
    #        torch.save(predictor.model.state_dict(), os.path.join('../models', FINE_TUNED_MODEL))

    #    if step == 1:
    #        mean_iou = 0

    #    mean_iou = mean_iou * 0.99 + 0.01 * np.mean(iou.cpu().detach().numpy())

    #    if step % 10 == 0:
    #         print("Step " + str(step) + ":\t", "Accuracy (IoU) = ", mean_iou)


C:\Users\saksh\AppData\Local\Temp\ipykernel_25208\2944438641.py:6: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
c:\Users\saksh\OneDrive\Documents\Duke Academics\Spring 2025\Deep Learning\CV-Module-Project\.venv\Lib\site-packages\torch\optim\lr_scheduler.py:227: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Step 1:	 Accuracy (IoU) =  2.6692345272749662e-05
Step 2:	 Accuracy (IoU) =  3.642709052655846e-05
Step 3:	 Accuracy (IoU) =  4.4282271031988784e-05
Step 4:	 Accuracy (IoU) =  5.0469204274995716e-05
Step 5:	 Accuracy (IoU) =  5.761604488317098e-05


In [ ]:
# Initialize scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=500)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=500, gamma=0.2) # 500 , 250, gamma = 0.1
accumulation_steps = 2  # Number of steps to accumulate gradients before updating

train_losses = []
train_ious = []
val_losses = []
val_ious = []

mean_iou = 0

for step in range(1, NO_OF_STEPS + 1):
    with torch.cuda.amp.autocast():
        image, mask, input_point, num_masks = read_batch(train_data, visualize_data=False)
        if image is None or mask is None or num_masks == 0:
            continue

        input_label = np.ones((num_masks, 1))
        if not isinstance(input_point, np.ndarray) or not isinstance(input_label, np.ndarray):
            continue

        if input_point.size == 0 or input_label.size == 0:
           continue
        
        image = image.copy()
        predictor.set_image(image)
        mask_input, unnorm_coords, labels, unnorm_box = predictor._prep_prompts(
            input_point, input_label, box=None, mask_logits=None, normalize_coords=True
        )
        if unnorm_coords is None or labels is None or unnorm_coords.shape[0] == 0 or labels.shape[0] == 0:
            continue

        sparse_embeddings, dense_embeddings = predictor.model.sam_prompt_encoder(
            points=(unnorm_coords, labels), boxes=None, masks=None,
        )

        batched_mode = unnorm_coords.shape[0] > 1
        high_res_features = [feat_level[-1].unsqueeze(0) for feat_level in predictor._features["high_res_feats"]]
        low_res_masks, prd_scores, _, _ = predictor.model.sam_mask_decoder(
            image_embeddings=predictor._features["image_embed"][-1].unsqueeze(0),
            image_pe=predictor.model.sam_prompt_encoder.get_dense_pe(),
            sparse_prompt_embeddings=sparse_embeddings,
            dense_prompt_embeddings=dense_embeddings,
            multimask_output=True,
            repeat_image=batched_mode,
            high_res_features=high_res_features,
        )
        prd_masks = predictor._transforms.postprocess_masks(low_res_masks, predictor._orig_hw[-1])

        gt_mask = torch.tensor(mask.astype(np.float32)).cuda()
        prd_mask = torch.sigmoid(prd_masks[:, 0])

        # Compute loss
        criterion = torch.nn.BCEWithLogitsLoss()

        if gt_mask.shape[0] == 1 and prd_masks.shape[0] > 1:
            gt_mask = gt_mask.expand(prd_masks.shape[0], -1, -1)

        seg_bce_loss = criterion(prd_masks[:, 0], gt_mask)
        seg_dice_loss = dice_loss(gt_mask, prd_masks[:, 0])
        seg_loss = seg_bce_loss + seg_dice_loss

        inter = (gt_mask * (prd_mask > 0.5)).sum(1).sum(1)
        iou = inter / (gt_mask.sum(1).sum(1) + (prd_mask > 0.5).sum(1).sum(1) - inter)
        score_loss = torch.abs(prd_scores[:, 0] - iou).mean()
        loss = seg_loss + score_loss * 0.05

        # Apply gradient accumulation
        loss = loss / accumulation_steps
        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(predictor.model.parameters(), max_norm=1.0)

        if step % accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            predictor.model.zero_grad()

        scheduler.step()

        if step % 500 == 0:
            FINE_TUNED_MODEL = FINE_TUNED_MODEL_NAME + "_" + str(step) + ".torch"
            torch.save(predictor.model.state_dict(), os.path.join('../models', FINE_TUNED_MODEL))

        mean_iou = mean_iou * 0.99 + 0.01 * np.mean(iou.cpu().detach().numpy())

        if step % 10 == 0:
            print(f"Step {step}:\t Accuracy (IoU) = {mean_iou:.4f}")
            train_losses.append(loss.item())  # Store full loss
            train_ious.append(mean_iou)       # Store running mean IoU

        # ---- Run Validation Every 100 Steps ----
        if step % 100 == 0:
            val_loss, val_iou = validate_model(predictor, val_data)
            val_losses.append(val_loss)
            val_ious.append(val_iou)

C:\Users\saksh\AppData\Local\Temp\ipykernel_25208\3045340707.py:14: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
c:\Users\saksh\OneDrive\Documents\Duke Academics\Spring 2025\Deep Learning\CV-Module-Project\.venv\Lib\site-packages\torch\optim\lr_scheduler.py:227: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Step 1:	 Accuracy (IoU) = 0.0000
Step 2:	 Accuracy (IoU) = 0.0000
Validation Loss: 19.6326, Validation IoU: 0.0013
Step 3:	 Accuracy (IoU) = 0.0000
Step 4:	 Accuracy (IoU) = 0.0000
Validation Loss: 19.7637, Validation IoU: 0.0012
Step 5:	 Accuracy (IoU) = 0.0001


In [ ]:
import json

with open("../models/training_data.json", "w") as f:
    json.dump({"train_losses": train_losses, "train_ious": train_ious, 
               "val_losses": val_losses, "val_ious": val_ious}, f)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Load training data if saved previously
# data = json.loads("..models/training_data.json")
# train_losses = data["train_losses"]
# train_ious = data["train_ious"]
# val_losses = data["val_losses"]
# val_ious = data["val_ious"]

# Plot Loss Curves
plt.figure(figsize=(12, 5))

# Plot Training vs Validation Loss
plt.subplot(1, 2, 1)
plt.plot(range(len(train_losses)), train_losses, label="Training Loss", color="blue")
plt.plot(range(0, len(train_losses), len(train_losses)//len(val_losses)), val_losses, label="Validation Loss", color="red", linestyle="dashed")
plt.xlabel("Steps")
plt.ylabel("Loss")
plt.title("Training & Validation Loss")
plt.legend()
plt.grid()

# Plot IoU Scores
plt.subplot(1, 2, 2)
plt.plot(range(len(train_ious)), train_ious, label="Training IoU", color="green")
plt.plot(range(0, len(train_ious), len(train_ious)//len(val_ious)), val_ious, label="Validation IoU", color="orange", linestyle="dashed")
plt.xlabel("Steps")
plt.ylabel("IoU Score")
plt.title("Training & Validation IoU")
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()


# Inference

In [8]:

def read_image(image_path, mask_path):  
    img = cv2.imread(image_path)[..., ::-1]  # Convert BGR to RGB
    mask = cv2.imread(mask_path, 0)

    r = min(1024 / img.shape[1], 1024 / img.shape[0])
    img = cv2.resize(img, (int(img.shape[1] * r), int(img.shape[0] * r)))
    mask = cv2.resize(mask, (int(mask.shape[1] * r), int(mask.shape[0] * r)), interpolation=cv2.INTER_NEAREST)

    return img, mask

def get_points(mask, num_points=1000):
   points = []
   coords = np.argwhere(mask > 0)
   for i in range(num_points):
       yx = np.array(coords[np.random.randint(len(coords))])
       points.append([[yx[1], yx[0]]])
   return np.array(points)

def process_img(image_path):  
    img = cv2.imread(image_path)[..., ::-1]  # Convert BGR to RGB

    r = min(1024 / img.shape[1], 1024 / img.shape[0])
    img = cv2.resize(img, (int(img.shape[1] * r), int(img.shape[0] * r)))
    return img


def get_random_points(image_shape, num_points=1000):  
    points = np.random.randint(0, [image_shape[1], image_shape[0]], size=(num_points, 1, 2))
    return points

In [9]:
# Load the fine-tuned model
FINE_TUNED_MODEL_WEIGHTS = "../models/fine_tuned_sam2_400_training_1000.torch"
# FINE_TUNED_MODEL_WEIGHTS = "../models/fine_tuned_sam2_only_anemic_1000.torch"

# Build net and load weights
predictor_fine_tuned = SAM2ImagePredictor(sam2_model)
predictor_fine_tuned.model.load_state_dict(torch.load(FINE_TUNED_MODEL_WEIGHTS))

<All keys matched successfully>

In [10]:
# FINE-TUNED
def run_inference(image_path, mask_path):
    # Load the selected image and mask
    image, mask = read_image(image_path, mask_path)

    # Generate random points for the input
    input_points = get_points(mask)

    # Perform inference and predict masks
    with torch.no_grad():
        image= image.copy()
        predictor_fine_tuned.set_image(image)
        masks, scores, logits = predictor_fine_tuned.predict(
            point_coords=input_points,
            point_labels=np.ones([input_points.shape[0], 1])
        )

    # Process the predicted masks and sort by scores
    np_masks = np.array(masks[:, 0])
    np_scores = scores[:, 0]
    sorted_masks = np_masks[np.argsort(np_scores)][::-1]

    seg_map = 1 - sorted_masks[0]

    return image, mask, seg_map


def run_inference_without_ground_truth(image_path):
    # Load the selected image and mask
    image = process_img(image_path)

    input_points = get_random_points(image.shape)

    # Perform inference and predict masks
    with torch.no_grad():
        image = image.copy()
        predictor_fine_tuned.set_image(image)
        masks, scores, logits = predictor_fine_tuned.predict(
            point_coords=input_points,
            point_labels=np.ones([input_points.shape[0], 1])
        )

    # Process the predicted masks and sort by scores
    np_masks = np.array(masks[:, 0])
    np_scores = scores[:, 0]
    sorted_masks = np_masks[np.argsort(np_scores)][::-1]

    seg_map = 1 - sorted_masks[0]

    return image, seg_map


In [11]:
# OUT OF THE BOX MODEL
def run_inference_original_model_with_prompt(image_path):
    # Load the selected image and mask
    image = process_img(image_path)

    input_points = get_random_points(image.shape)

    # Perform inference and predict masks
    with torch.no_grad():
        image = image.copy()
        predictor.set_image(image)
        masks, scores, logits = predictor.predict(
            point_coords=input_points,
            point_labels=np.ones([input_points.shape[0], 1])
        )

    # Process the predicted masks and sort by scores
    np_masks = np.array(masks[:, 0])
    np_scores = scores[:, 0]
    sorted_masks = np_masks[np.argsort(np_scores)][::-1]

    seg_map = 1 - sorted_masks[0]

    return image, seg_map

def run_inference_with_automatic_mask_generator(image_path):
    from sam2.automatic_mask_generator import SAM2AutomaticMaskGenerator
    
    image = process_img(image_path)

    sam2 = build_sam2(model_cfg, sam2_checkpoint, device=torch.device("cuda"), apply_postprocessing=False)
    sam2_mask_generator = SAM2AutomaticMaskGenerator(sam2)

    masks = sam2_mask_generator.generate(image)
    masks = sorted(masks, key=lambda x: x['predicted_iou'], reverse=True)

    print(masks[0])

    return image, 1 - masks[0]['segmentation']


In [12]:
def visualize_results(image, ground_truth_mask, predicted_mask):
    """Visualizes the original image, ground truth mask, and predicted mask side by side."""
    
    # Create a figure with 3 subplots
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Display the original image
    axes[0].imshow(image, cmap='gray')
    axes[0].set_title('Original Image')
    axes[0].axis('off')
    
    # Display the ground truth mask
    axes[1].imshow(ground_truth_mask, cmap='gray')
    axes[1].set_title('Ground Truth Mask')
    axes[1].axis('off')
    
    # Display the predicted segmentation mask
    axes[2].imshow(predicted_mask, cmap='gray')
    axes[2].set_title('Predicted Mask')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()

In [13]:
import random

selected_entry = random.choice(test_data)
image_path = selected_entry['image']
mask_path = selected_entry['annotation']

print(image_path)

../data/raw/AneRBC-I/Anemic_individuals/Original_images\097_a.png


In [14]:
# # Hangs my system, takes ~ 5 mins to run
# mask = process_img(mask_path)
# image, seg_map = run_inference_with_automatic_mask_generator(image_path)
# visualize_results(image, mask, seg_map)

In [15]:
# # Perform inference and visualize the results
# mask = process_img(mask_path)
# image, mask, seg_map = run_inference_original_model_with_prompt(image_path)
# visualize_results(image, mask, seg_map)

In [16]:
# Perform inference and visualize the results
image, mask, seg_map = run_inference(image_path, mask_path)
visualize_results(image, mask, seg_map)

OutOfMemoryError: CUDA out of memory. Tried to allocate 3.91 GiB. GPU 0 has a total capacity of 6.00 GiB of which 0 bytes is free. Of the allocated memory 12.08 GiB is allocated by PyTorch, and 34.91 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [16]:
# Function to compute Pixel Accuracy
def pixel_accuracy(y_true, y_pred):
    correct_pixels = np.sum(y_true == y_pred)
    total_pixels = np.prod(y_true.shape)
    return correct_pixels / (total_pixels)

# Function to compute IoU (Intersection over Union)
def iou(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    union = np.sum(y_true) + np.sum(y_pred) - intersection
    return intersection / (union)

# Function to compute Dice Coefficient
def dice_coef(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    return (2. * intersection) / (np.sum(y_true) + np.sum(y_pred))

# Function to compute Precision
def precision(y_true, y_pred):
    true_positives = np.sum((y_true == 1) & (y_pred == 1))
    predicted_positives = np.sum(y_pred == 1)
    return true_positives / (predicted_positives)

# Function to compute Recall
def recall(y_true, y_pred):
    true_positives = np.sum((y_true == 1) & (y_pred == 1))
    actual_positives = np.sum(y_true == 1)
    return true_positives / (actual_positives)

# Function to compute Specificity
def specificity(y_true, y_pred):
    true_negatives = np.sum((y_true == 0) & (y_pred == 0))
    possible_negatives = np.sum(y_true == 0)
    return true_negatives / (possible_negatives)

# Function to compute Dice Loss
def dice_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

# Example: Calculate metrics for a given test case
def calculate_metrics(mask, seg_map):
    # Convert mask and segmentation map to binary (0 or 1)
    mask_binary = (mask > 0).astype(np.uint8)
    seg_map_binary = (seg_map > 0).astype(np.uint8)

    # Calculate each metric
    pixel_acc = pixel_accuracy(mask_binary, seg_map_binary)
    iou_score = iou(mask_binary, seg_map_binary)
    dice_score = dice_coef(mask_binary, seg_map_binary)
    precision_score = precision(mask_binary, seg_map_binary)
    recall_score = recall(mask_binary, seg_map_binary)
    specificity_score = specificity(mask_binary, seg_map_binary)
    loss = dice_loss(mask_binary, seg_map_binary)

    return pixel_acc, iou_score, dice_score, precision_score, recall_score, specificity_score, loss


# Calculate metrics for this sample
pixel_acc, iou_score, dice_score, precision_score, recall_score, specificity_score, loss = calculate_metrics(mask, seg_map)
print(f"Pixel Accuracy: {pixel_acc:.4f}")
print(f"IoU: {iou_score:.4f}")
print(f"Dice Coefficient: {dice_score:.4f}")
print(f"Precision: {precision_score:.4f}")
print(f"Recall: {recall_score:.4f}")
print(f"Specificity: {specificity_score:.4f}")
print(f"Dice Loss: {loss:.4f}")


NameError: name 'mask' is not defined

In [ ]:
image, seg_map = run_inference_without_ground_truth(image_path)
visualize_results(image, mask, seg_map)
mask = np.where(mask > 128, 255, 0).astype(np.uint8)

pixel_acc, iou_score, dice_score, precision_score, recall_score, 
specificity_score, loss = calculate_metrics(mask, seg_map)
print(f"Pixel Accuracy: {pixel_acc:.4f}")
print(f"IoU: {iou_score:.4f}")
print(f"Dice Coefficient: {dice_score:.4f}")
print(f"Precision: {precision_score:.4f}")
print(f"Recall: {recall_score:.4f}")
print(f"Specificity: {specificity_score:.4f}")
print(f"Dice Loss: {loss:.4f}")

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [12]:
mean_pixel_accuracy = []
mean_iou_acc = []
mean_dc = []
mean_pre = []
mean_recall = []
mean_speci = []
mean_dl = []

for test_img in tqdm(test_data):
    image_path = test_img['image']
    mask_path = test_img['annotation']

    image, mask, seg_map = run_inference(image_path, mask_path)
    
    pixel_acc, iou_score, dice_score, precision_score, recall_score, specificity_score, loss = calculate_metrics(mask, seg_map)

    mean_pixel_accuracy.append(pixel_acc)
    mean_iou_acc.append(iou_score)
    mean_dc.append(dice_score)
    mean_pre.append(precision_score)
    mean_recall.append(recall_score)
    mean_speci.append(specificity_score)
    mean_dl.append(loss)

100%|██████████| 600/600 [17:00<00:00,  1.70s/it]


In [13]:
print(f"Mean Pixel Accuracy for test data: {np.mean(mean_pixel_accuracy):.4f}")
print(f"Mean IoU for test data: {np.mean(mean_iou_acc):.4f}")
print(f"Mean Dice Coefficient for test data: {np.mean(mean_dc):.4f}")
print(f"Mean Precision for test data: {np.mean(mean_pre):.4f}")
print(f"Mean Recall for test data: {np.mean(mean_recall):.4f}")
print(f"Mean Specificity for test data: {np.mean(mean_speci):.4f}")
print(f"Mean Dice Loss for test data: {np.mean(mean_dl):.4f}")

Mean Pixel Accuracy for test data: 0.6123
Mean IoU for test data: 0.6119
Mean Dice Coefficient for test data: 0.7567
Mean Specificity for test data: 0.5650
Mean Dice Loss for test data: 0.2433


In [ ]:
mean_pixel_accuracy = []
mean_iou_acc = []
mean_dc = []
mean_pre = []
mean_recall = []
mean_speci = []
mean_dl = []

for test_img in tqdm(test_data):
    image_path = test_img['image']
    mask_path = test_img['annotation']

    mask = process_img(mask_path)
    image, seg_map = run_inference_without_ground_truth(image_path, mask_path)
    pixel_acc, iou_score, dice_score, precision_score, recall_score, specificity_score, loss = calculate_metrics(mask, seg_map)

    mean_pixel_accuracy.append(pixel_acc)
    mean_iou_acc.append(iou_score)
    mean_dc.append(dice_score)
    mean_pre.append(precision_score)
    mean_recall.append(recall_score)
    mean_speci.append(specificity_score)
    mean_dl.append(loss)

In [ ]:
print(f"Mean Pixel Accuracy for test data: {np.mean(mean_pixel_accuracy):.4f}")
print(f"Mean IoU for test data: {np.mean(mean_iou_acc):.4f}")
print(f"Mean Dice Coefficient for test data: {np.mean(mean_dc):.4f}")
print(f"Mean Precision for test data: {np.mean(mean_pre):.4f}")
print(f"Mean Recall for test data: {np.mean(mean_recall):.4f}")
print(f"Mean Specificity for test data: {np.mean(mean_speci):.4f}")
print(f"Mean Dice Loss for test data: {np.mean(mean_dl):.4f}")